# Implement Multi Agent Framework

In [2]:
!pip install python-dotenv --quiet
!pip install tenacity --quiet
!pip install openai --quiet

In [3]:
from openai import OpenAI
from typing import Optional
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import List, Optional, Dict
import logging

### LLM Preparation 

In [4]:
# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Agent & Multi-Agent 

In [5]:

class BaseAgent:
    """
    An example of a base agent class, including an OpenAI client.
    """
    def __init__(self,client:OpenAI):
        self.client = client
        # can add more attributes and methods here
        

class Agent:
    """
    A class representing an agent that can handle chat messages and handoff to other agents.
    """
    def __init__(
            self,
            base_agent:BaseAgent,
            name:str,
            instructions: Optional[str] = None):
        self.DEFAULT_MODEL = "gpt-4o-mini"
        self.name = name.replace(" ","_")
        self.agent = base_agent
        self.instructions = instructions
        self.handoff_agent_schemas: List[Dict] = []
        self.handoff_agents: Dict[str, "Agent"] = {}

    def __str__(self):
        """
        Returns a string representation of the Agent instance.
        """
        return f"Agent(name={self.name}, instructions={self.instructions},handoff_agents={list(self.handoff_agents.keys())})"


    def add_handoffs(self, agents: List["Agent"]) -> None:
        """ 
        Adds handoff agents to the current agent.
        """
        self.handoff_agent_schemas.clear()
        for agent in agents:
            agent_schema = {
                'type': 'function',
                'function': {
                    'name': agent.name,
                    'description': agent.instructions or '',
                    'parameters': {'type': 'object', 'properties': {}, 'required': []}
                }
            }
            self.handoff_agent_schemas.append(agent_schema)
            self.handoff_agents[agent.name] = agent

    @retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
    def chat(self,messages:List[Dict],model: Optional[str] = None,disable_tools:bool = False):
        """ 
        Handles chat messages and returns responses, with optional handoff to other agents.
        """

        if self.instructions:
            messages = [{"role": "system", "content": self.instructions}] + messages
        
        model = model or self.DEFAULT_MODEL

        try:

            response = self.agent.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=None if disable_tools else self.handoff_agent_schemas or None,
                tool_choice=None,
            )
            message = response.choices[0].message
            if not message.tool_calls:
                result = [{"role": "assistant", "content": message.content, "agent_name": self.name}]
                return result
            
            result = []
            for tool_call in message.tool_calls:
                call_agent = self.handoff_agents[tool_call.function.name]
                call_message = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "handoff": f"{self.name} -> {call_agent.name}",
                    "agent_name": call_agent.name,
                    "agent": call_agent,
                }
                result.append(call_message)      
            return result
        except Exception as e:
            logging.error("Unable to generate ChatCompletion response")
            logging.error(f"Exception: {e}")
            return None


class MultiAgent:
    def __init__(self,start_agent:Optional[Agent] = None):
        """
        Initialize the MultiAgent with an optional starting agent.
        """
        self.current_agent = start_agent

    def add_handoff_relations(self,from_agent:Agent,to_agents:List[Agent])->None:
        """
        Add handoff relations from one agent to multiple agents.
        """
        from_agent.add_handoffs(to_agents)

    def chat(self,messages:List[Dict],model: Optional[str] = None,agent:Optional[Agent] = None,max_handoof_depth:int = 2)->List[Dict]:
        """
        Chat with the current agent or a specified agent using the provided messages and model.
        """
        if agent:
            self.current_agent = agent

        if not self.current_agent:
            logging.error("No agent is set for conversation. You can set it in init or pass as argument.")
            return None
        
        try:
            # currently only one handoff is supported
            res = [self.current_agent.chat(messages, model)[0]]
            if res and res[0].get("content"):
                return res
            while res and res[-1].get("tool_call_id") and max_handoof_depth > 0:
                max_handoof_depth -= 1
                self.current_agent = res[-1]["agent"]
                if max_handoof_depth == 0:
                    res.extend(self.current_agent.chat(messages, model, disable_tools=True))
                else:
                    res.extend(self.current_agent.chat(messages, model))
            return res
        except Exception as e:
            logging.error("Error during chat with agent.")
            logging.error(f"Exception: {e}")
            return None
        
    def handoff(self,messages:List[Dict],model: Optional[str] = None,agent:Agent = None)-> Agent:
        """
        Perform a handoff to the specified agent using the provided messages and model.
        """
        current_agent = agent

        if not current_agent:
            logging.error("No agent is set for conversation. You must specify an agent.")
            return None
        
        try:
            res = current_agent.chat(messages, model)
            handoff_agents = {}
            if res:
                for r in res:
                    if r.get("tool_call_id"):
                        handoff_agents[r["agent_name"]] = r["agent"]
            if handoff_agents:
                return handoff_agents
            return None
        except Exception as e:
            logging.error("Error during handoff with agent.")
            logging.error(f"Exception: {e}")
            return None

### Init Several Agents

In [10]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Agent used for daily questions named Ada.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent used for science questions named Albert.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent used for music questions named Mozart.")

###  Build a Multi-Agent environment & add handoff relationships 

In [11]:
ma = MultiAgent(start_agent = general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent])

# Multi-Agent handoff ability display

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [12]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x112f22010>}

In [13]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x112f22010>,
 'music_agent': <__main__.Agent at 0x112f44fd0>}

### Multi-Agent dialogue ability display

In [14]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'tool',
  'tool_call_id': 'call_yjmTOZ4eMn6w3SHV2myrZxkz',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <__main__.Agent at 0x112f22010>},
 {'role': 'assistant',
  'content': "The sky appears blue due to a phenomenon called Rayleigh scattering. Here's how it works:\n\n1. **Sunlight and Atmosphere**: Sunlight, or white light, is made up of different colors, each with different wavelengths. When sunlight enters the Earth's atmosphere, it interacts with air molecules and small particles.\n\n2. **Scattering of Light**: Shorter wavelengths of light (blue and violet) scatter more than longer wavelengths (red and yellow) when they strike these particles. Although violet light scatters even more than blue, our eyes are more sensitive to blue light and some of the violet light is absorbed by the ozone layer.\n\n3. **Perception of Color**: Because blue light is scattered in all directions, it is the dominant color we see when we look up at t

In [15]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, your AI assistant for science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]

In [16]:
ma.chat(messages=[{"role": "user", "content": "What are the different music styles?"}])

[{'role': 'tool',
  'tool_call_id': 'call_6h1Ds5jC798hoqGFocgofgAc',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x112f22e90>},
 {'role': 'tool',
  'tool_call_id': 'call_X67JuGgyd5aeZbvQywM004n6',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <__main__.Agent at 0x112f44fd0>},
 {'role': 'assistant',
  'content': 'Music styles encompass a wide range of genres and subgenres, each with its own unique characteristics and cultural influences. Here are some of the major music styles:\n\n1. **Classical**: Characterized by a structured form, orchestral arrangements, and complex compositions. It includes periods such as Baroque, Classical, Romantic, and Contemporary.\n\n2. **Jazz**: Known for its swing and blue notes, jazz emphasizes improvisation and includes styles like bebop, swing, and smooth jazz.\n\n3. **Rock**: Evolving from 1950s rock and roll, it includes numerous subgenres like cl

In [17]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'tool',
  'tool_call_id': 'call_LtMblVxNr2FVPiCJicAtaji9',
  'handoff': 'music_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x112f22e90>},
 {'role': 'assistant',
  'content': 'What are you in the mood for? Any specific cuisine or ingredients you want to include in your dinner?',
  'agent_name': 'general_agent'}]

In [18]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': 'I am Albert, your assistant for science questions. How can I help you today?',
  'agent_name': 'science_agent'}]